In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-13"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6405,2024-04-13,Brasil Nbb,17:00,Cerrado,Fortaleza B.C.,2.09,1.69,153.5,1.85,1.85,1.5,1.94,1.54,44LFjEUb,0.478469,0.591716,0.540541,0.540541,0.070185,0.6,0.4,NaN,NaN,247.952,115.821339,0.467112,120.102,26.430744,0.220069,245.120,237.414,430.77,155.31,1.0,0.0,0.0,0.0,0.149652,0.000000,0.162553,-1.98,-0.396,-2.752525,0.433972,0.4,-0.033972,-0.36,-0.072,-9.583333,0.403033,0.3,-0.103033
6406,2024-04-13,Espanha Acb,13:00,Obradoiro CAB,Manresa,1.95,1.86,167.5,1.89,1.89,1.5,1.82,1.71,UocQbz9U,0.512821,0.537634,0.529101,0.529101,0.050455,0.8,0.4,NaN,NaN,268.200,85.077300,0.317216,175.550,39.764207,0.226512,231.786,315.226,355.32,170.52,1.0,0.0,0.0,0.0,0.033407,0.000000,0.044069,-5.00,-1.000,-0.950000,0.000000,0.0,0.000000,7.70,1.540,0.558442,0.000000,0.0,0.000000
6407,2024-04-13,Itália Liga A,15:00,Trento,Tortona,1.87,1.94,163.5,1.85,1.93,-1.5,1.97,2.04,OYFavOE9,0.534759,0.515464,0.540541,0.518135,0.050223,0.8,0.6,NaN,NaN,161.586,50.057577,0.309789,136.406,18.176504,0.133253,149.196,196.252,148.77,154.70,0.0,0.0,0.0,0.0,0.025983,0.029930,0.024687,-1.84,-0.368,-2.364130,0.000000,0.0,0.000000,-3.28,-0.656,-1.432927,0.462914,0.3,-0.162914
6408,2024-04-13,Itália Liga A,15:30,Pesaro,Pistoia,1.84,1.97,166.5,1.85,1.91,-1.5,1.97,2.07,zDZ4Ithj,0.543478,0.507614,0.540541,0.523560,0.051092,0.2,1.0,NaN,NaN,214.222,75.087153,0.350511,196.738,65.464695,0.332751,162.234,297.232,140.76,269.10,0.0,0.0,0.0,0.0,0.048254,0.022567,0.035005,-5.00,-1.000,-0.840000,0.517902,0.2,-0.317902,0.98,0.196,4.948980,0.000000,0.0,0.000000
6409,2024-04-13,Alemanha Pro A,14:00,Giessen,Phoenix Hagen,1.70,2.07,170.5,1.83,1.87,-2.5,1.93,2.46,tIWrWLn5,0.588235,0.483092,0.546448,0.534759,0.071327,0.4,0.6,NaN,NaN,136.618,27.077055,0.198195,122.870,11.191394,0.091083,123.600,141.476,102.86,113.24,0.0,0.0,0.0,0.0,0.138795,0.015289,0.170736,-0.74,-0.148,-4.729730,0.000000,0.0,0.000000,0.31,0.062,17.258065,0.000000,0.0,0.000000
6410,2024-04-13,Alemanha Pro A,14:30,Koblenz,Bayreuth,2.03,1.71,157.5,1.81,1.85,1.5,1.90,1.57,AcOaSwnU,0.492611,0.584795,0.552486,0.540541,0.077406,0.4,0.6,NaN,NaN,213.696,70.588845,0.330324,137.964,24.604863,0.178343,187.538,112.380,232.98,113.52,0.0,0.0,0.0,0.0,0.121002,0.015456,0.134493,0.84,0.168,6.130952,0.000000,0.0,0.000000,-5.00,-1.000,-0.710000,0.000000,0.0,0.000000
6411,2024-04-13,Alemanha Pro A,14:30,Munster,Kirchheim,2.01,1.73,163.5,1.84,1.86,1.5,1.85,1.61,GzPeTJWN,0.497512,0.578035,0.543478,0.537634,0.075547,0.6,0.6,NaN,NaN,206.148,109.031362,0.528898,188.444,56.893689,0.301913,130.724,229.888,131.40,207.48,0.0,0.0,0.0,0.0,0.105877,0.007644,0.098096,-0.63,-0.126,-8.015873,0.000000,0.0,0.000000,0.35,0.070,10.428571,0.000000,0.0,0.000000
6412,2024-04-13,Argentina Liga A,21:00,Argentino,Obera TC,2.10,1.68,154.5,1.83,1.87,1.5,1.95,1.54,rDcK3B3K,0.476190,0.595238,0.546448,0.534759,0.071429,0.2,0.8,NaN,NaN,314.530,199.499666,0.634279,142.374,67.047704,0.470927,174.218,205.282,156.42,106.08,0.0,0.0,0.0,0.0,0.157135,0.015289,0.166140,0.86,0.172,6.395349,0.423785,0.5,0.076215,2.80,0.560,1.214286,0.000000,0.0,0.000000
6413,2024-04-13,Austrália Nbl1 Leste,06:30,Albury-Wodonga Bandits,Sutherland,1.78,1.94,173.5,1.85,1.85,-1.5,1.89,2.06,KUfBqIeF,0.561798,0.515464,0.540541,0.540541,0.077262,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.060826,0.000000,0.060865,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6414,2024-04-13,Aus

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,15:00,Itália Liga A,Trento,Tortona,163.5,1.85,1.0000,Over
1,15:30,Itália Liga A,Pesaro,Pistoia,166.5,1.85,1.0000,Over
2,14:00,Alemanha Pro A,Giessen,Phoenix Hagen,170.5,1.83,1.0000,Over
3,06:30,Austrália Nbl1 Leste,Albury-Wodonga Bandits,Sutherland,173.5,1.85,0.9987,Over
4,06:30,Austrália Nbl1 Leste,Sydney Comets F,Central Coast F,151.5,1.83,1.0000,Over
5,06:15,Austrália Nbl1 Norte,Ipswich Force,Gold Coast Rollers,191.5,1.82,1.0000,Over
6,07:00,Austrália Nbl1 Sul,Hobart Chargers,Waverley,166.5,1.83,0.9999,Over
7,04:30,Austrália Nbl1 Sul,Melbourne Tigers F,Ringwood F,153.5,1.86,1.0000,Over
8,09:00,Austrália Nbl1 Oeste,Goldfields Giants,Joondalup Wolves,177.5,1.85,1.0000,Over
9,09:30,Irã Superliga,Kalleh,Zob Ahan,160.5,1.83,0.9997,Over
